In [7]:
import streamlit as st
from dotenv import load_dotenv
import pandas as pd
from os import getenv
import psycopg2
import requests, json
from datetime import datetime
from psycopg2.extras import execute_values
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import os



def weather_data(lat, lon, key):
    """Obtains weather data from the OpenWeatherMap API for the next five days and stores it in a DataFrame."""


    key = '4f44009207da925ce842972729de1b50'
    url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={key}'
    response = requests.get(url)
    
    if response.ok:
        data = response.json()
        
        data_frame = []

        city_name = data['city']['name']

        # Iterate through the list of weather data
        for dct in data['list']:
            timestamp = dct['dt']  # dt = unix time
            hour = datetime.fromtimestamp(timestamp)
            temp_min = dct["main"]["temp_min"]
            temp_max = dct["main"]["temp_max"]
            temperature = dct["main"]["temp"]
            humidity = dct["main"]["humidity"]
            air_pressure = dct["main"]["pressure"]
            day_category = dct["weather"][0]["main"]
            current_status = dct["weather"][0]["description"]
            speed_of_wind = dct["wind"]["speed"]
            visibility = dct.get("visibility", "0")  # Visibility may not always be available
            pop = dct.get("pop", "0")  # Pop may not always be available
            sys_info = dct["sys"]["pod"]

            # Append the data for iteration to the list
            data_frame.append([city_name, timestamp, hour, temp_min, temp_max, temperature, humidity, air_pressure, day_category, current_status, speed_of_wind, visibility, pop, sys_info])

        return data_frame
    else:
        print("Failed to retrieve weather data.")
        return None
    

def retrieve_weather_data_for_counties(counties, key):
    counties = pd.read_csv(r'C:\Users\jjs61\OneDrive\Desktop\Streamlit\src\data\us-county-boundaries.csv')
    all_weather_data = []  # Initialize an empty list to store the weather data

    for _, row in counties.iterrows():
        lat, lon = row['latitude'], row['longitude']
        weather_data_result = weather_data(lat, lon, key)
        
        if weather_data_result is not None:
            for data in weather_data_result:
                data.extend([lat, lon])  # Append latitude and longitude
            all_weather_data.extend(weather_data_result)
    
    return all_weather_data


counties = pd.read_csv(r'C:\Users\jjs61\OneDrive\Desktop\Streamlit\src\data\us-county-boundaries.csv')
key = 'bf5bec80bffcd85b43121653e17a6c4a'
# Call function to retrieve weather data for locations in the counties DataFrame
all_weather_data = retrieve_weather_data_for_counties(counties, key)

# Create a DataFrame from the combined weather data

In [11]:
# # Create a DataFrame from the combined weather data
columns = ['city', 'timestamp', 'hour', 'temp_min', 'temp_max', 'temperature', 'humidity', 'air_pressure', 'day_category', 'current_status', 'speed_of_wind', 'visibility_in_meters', 'pop', 'sys', 'latitude', 'longitude']

df_weather = pd.DataFrame(all_weather_data, columns=columns)
df_weather

,city,timestamp,hour,temp_min,temp_max,temperature,humidity,air_pressure,day_category,current_status,speed_of_wind,visibility_in_meters,pop,sys,latitude,longitude
0,West Union,1698861600,2023-11-01 14:00:00,277.24,279.93,277.24,57,1030,Clear,clear sky,3.61,10000,0.0,d,38.845515,-83.472152
1,West Union,1698872400,2023-11-01 17:00:00,278.45,279.72,278.45,47,1029,Clear,clear sky,2.84,10000,0.0,d,38.845515,-83.472152
2,West Union,1698883200,2023-11-01 20:00:00,273.91,273.91,273.91,57,1029,Clear,clear sky,0.39,10000,0.0,n,38.845515,-83.472152
3,West Union,1698894000,2023-11-01 23:00:00,272.67,272.67,272.67,63,1030,Clear,clear sky,1.39,10000,0.0,n,38.845515,-83.472152
4,West Union,1698904800,2023-11-02 02:00:00,271.80,271.80,271.80,67,1031,Clear,clear sky,1.40,10000,0.0,n,38.845515,-83.472152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3515,Upper Sandusky,1699239600,2023-11-05 22:00:00,285.23,285.23,285.23,87,1015,Clouds,overcast clouds,2.81,10000,0.0,n,40.842364,-83.304378
3516,Upper Sandusky,1699250400,2023-11-06 01:00:00,283.85,283.85,283.85,93,1015,Clouds,overcast clouds,2.74,10000,0.0,n,40.842364,-83.304378
3517,Upper Sandusky,1699261200,2023-11-06 04:00:00,284.05,284.05,284.05,91,1013,Clouds,overcast clouds,2.79,10000,0.0,n,40.842364,-83.304378
3518,Upper Sandusky,1699272000,2023-11-06 07:00:00,282.56,282.56,282.56,95,1012,Clouds,overcast clouds,3.53,10000,0.0,n,40.842364,-83.304378


In [9]:
import streamlit as st
from dotenv import load_dotenv
import pandas as pd
from os import getenv
import psycopg2
import requests, json
from datetime import datetime
from psycopg2.extras import execute_values
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import os

In [14]:
df_weather.index.rename('index', inplace=True)
# Convert temperature columns from Kelvin to Fahrenheit
df_weather['temp_min'] = ((df_weather['temp_min'] - 273.15) * 9/5 + 32)
df_weather['temp_max'] = ((df_weather['temp_max'] - 273.15) * 9/5 + 32)
df_weather['temperature'] = ((df_weather['temperature'] - 273.15) * 9/5 + 32)

# Define the split_timestamp function
def split_timestamp(timestamp):
    date = timestamp.date()
    hour = timestamp.time()
    return pd.Series([date, hour])

# Split the timestamp into date and hour columns
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'], unit='s')
df_weather[['date', 'hour']] = df_weather['timestamp'].apply(split_timestamp)

# Convert visibility from meters to miles
df_weather['visibility_in_miles'] = df_weather['visibility_in_meters'] * 0.000621371

# Convert wind speed from meters per second to feet per second
df_weather['speed_of_wind_feet_per_sec'] = df_weather['speed_of_wind'] * 3.28084

# Convert the 'timestamp' column to a datetime object
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'], unit='s')

In [2]:
load_dotenv()
url=getenv('SQL_URL')
df_weather.to_sql('weather_data', con = url, if_exists='replace', index=False)

NameError: name 'load_dotenv' is not defined

In [1]:
df_weather

NameError: name 'df_weather' is not defined